# Description of this ChatBot program with NLP

## Prerequisites

I have used this website as a starting point for this chatbot:
<u>https://data-flair.training/blogs/python-chatbot-project/</u>

## Problems

There were many problems to run that on an actual system.

I have tested it on:

-   M2 Mac

-   Intel Mac

-   Windows 11 virtual machine (on M2 Mac)

-   Ubuntu VM on Intel Mac

-   Ubuntu VM on M2 Mac.

First I had the problem that all elements on the application window were
black except a wonderful Mac like Button in white and a scrollbar.

I found out that the Python version 3.8 was to old and so I updated to
3.11.

For that and globally to let run the program you have to install all
these libraries:

-   nltk

-   numpy

-   keras

-   pickle

-   json

-   tensorflow

-   geopy

-   simple_dwd_weatherforecast

with pip-install *??? (one or more of the libraries above, space
separated)*

Bevore you can use this installation command, you have to install pip
with this command:

**sudo apt install python3-pip**

You also need the tinker part to create the window view:

**sudo apt-get install python3-tk**

Then you could have a running program. But on a Mac it looks not like
expected. In the code the defined button has a green color. On a Mac it
has a white color because the library uses the system colors and so the
named background colors are ignored. Only to set the letter color is
working.

Before you can use this as a chatbot, you have to let run the

**train_chatbot.py**

which has some more challenges. These are marked in the comments at
training.array and SGD.


## Additional information
To train the NLP chatbot part the file intents.json is used. In this file the expected texts and their responses are defined. But this file is not only used by the training part. It is also used by the runtime program.

## Initialisation Part
First we load all the libraries we need.

-   nltk: Natural Language Toolkit; for all the language processing we want to do. https://www.nltk.org

-   numpy: Scientific computing; for some array handling. https://numpy.org

-   keras: Deepl learning library: Simplification of the use of Tensorflow. https://keras.io

-   pickle: Python object serialization: Conversion from or to a stream into object hierarchy. https://docs.python.org/3/library/pickle.html

-   json: JSON encoder and decoder: Find parts of Json Objects. https://docs.python.org/3/library/json.html

-   tensorflow: Neuronal Network library: Create and use a convolutional neuronal network. https://www.tensorflow.org

-   tkinter: Python interface to Tcl/Tk: Used to create the view. https://docs.python.org/3/library/tkinter.html

-   random: Python library to randomize data: To randomize training data. https://docs.python.org/3/library/random.html

-   geopy: Python library to find geo locations (https://www.tutorialspoint.com/how-to-get-the-longitude-and-latitude-of-a-city-using-python)

-   simple_dwd_weatherforecast: Library to find a weather: (https://snyk.io/advisor/python/simple-dwd-weatherforecast)

-   datetime: Library to calculate date times

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

#Creating GUI with tkinter
import tkinter as tk
from tkinter import *

import numpy as np
from keras.models import load_model
model   = load_model('chatbot_model.h5')
import random
intents = json.loads(open('intents.json').read())
words   = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# Import the required library for finding geo locations for cities
from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyChatbot")

# For the weather forecast the dwd (Deutscher Wetter Dfrom simple_dwd_weatherforecast import dwdforecast
from simple_dwd_weatherforecast import dwdforecast
from datetime import datetime, timedelta, timezone

## 1 Part NLP process
The first we do at a NLP process is to find out the words of the input sentences.<br>
This is done by the function clean_up_sentence(sentence) with the sentence as an input and an array of word as an output.<br>
First the sentence is splitt into words and the second function reduces the words to their stems.

The second function bow() with the sentence and the defined words as an input returns an array with the words marked with an 1 if the word is in the defined list.<br>
The bow function uses the clean_up_sentence function.

The third function is the predict_class() function with the sentence and the model as an input. This function uses the upper bow function to get the important words and with the model predict function it gets a probabilty for the meaning of the input from the model. The result is a list of tags with their probability. There is another filter so probabilities lower than .25 are ignored.

The fourth function getResponse() returns the response. The response is a random response from the responses defined for the tags. With this the response not every time the same.

The last function of the process is the function which is called by the user interface. It calls the predict_class function and uses the result from their to create the random response.

In [2]:
# We need a global variable to store possible locations
globLocation      = None
errorMessage      = ""
actWeatherStation = None
lastWeatherStation= None

# Weather functions
# Return the location information for a given city name
def getLocation(name):
    if name is None:
        return None
    try:
        location = geolocator.geocode(name)
    except:
        global errorMessage
        errorMessage = "Error while looking for location"
        return None
    return location 


# we check the content of the sentence if we can find a date and then we return it
def getDate(sentence):
    import re

    time_now = datetime.today()
    print(time_now)
    english_date = re.search(r'\d{4}-\d{2}-\d{2}', sentence)
    german_date  = re.search(r'\d{2}.\d{2}.\d{4}', sentence)
    us_date      = re.search(r'\d{4}/\d{2}/\d{2}', sentence)

    if "now" in sentence.lower():
        return time_now
    elif "today" in sentence.lower():
        return time_now
    elif "tomorrow" in sentence.lower():
        return datetime.today() + timedelta(days=+1)
    elif english_date is not None:
        date = datetime.strptime(english_date.group(), '%Y-%m-%d').date()
        return date
    elif german_date is not None:
        date = datetime.strptime(german_date.group(), '%d.%m.%Y').date()
        return date
    elif us_date is not None:
        date = datetime.strptime(us_date.group(), '%Y/%m/%d').date()
        return date
    return time_now


# Returns the weather (the temperature and the cloud coverage) for a location
# This is just an example. A lot of more values are possible.
def getWeather(location, time):
    if location is None:
        return None

    # Attention: There is a bug in the simple_dwd_weatherforecast library. If you call the next functions multiple times for the
    # same locationit crashes. I have sent a bug report to the developers on github.com.
    try: 
        next_weather_station = dwdforecast.get_nearest_station_id(location.latitude, location.longitude)
        if next_weather_station is not None:
            global lastWeatherStation
            global actWeatherStation
            print("Station ID found: ", next_weather_station)
            # Optimization if we already looked for the station
            if lastWeatherStation is None or not (lastWeatherStation == next_weather_station):
                actWeatherStation = dwdforecast.Weather(next_weather_station)
                lastWeatherStation = next_weather_station

            # print("Looking for weather at ", next_weather_station, " for time ", time)
            temperature    = round(actWeatherStation.get_forecast_data(dwdforecast.WeatherDataType.TEMPERATURE, time) - 273.16, 2)
            cloud_coverage = actWeatherStation.get_forecast_data(dwdforecast.WeatherDataType.CLOUD_COVERAGE, time)
            return temperature, cloud_coverage
        else:
            return None
    except Exception as e:
        global errorMessage
        errorMessage = "Error while looking for weather: " + str(e)
        print(errorMessage)
        return None, None


# NLTK functions
# return an array of words from the sentence
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    other_words = []
    bag = [0]*len(words)
    for s in sentence_words:
        # print(s)
        word_found = False
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                word_found = True
                if show_details:
                    print ("found in bag: %s" % w)
        if word_found == False:
            other_words.append(s)
            if show_details:
                print("Other word found %s" % s)
    return(np.array(bag), other_words)


# return a sorted tag list with their predictions
# Teststring: What is the weather at Altenstadt
def predict_class(sentence, model):
    # print("Anzahl Worte: ", len(words))
    # filter out predictions below a threshold
    p, other_words = bow(sentence, words, show_details=False)
    
    # we are looking for some locations
    # print(other_words)
    for c in other_words:
        global globLocation
        globLocation = getLocation(c)
        if globLocation is not None:
            # We have a location, so we stop here
            print("We have a location")
            break

    
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


# In this method we create the response and if we know a location we give back the weather for that location
def getResponse(ints, intents_json, date):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            if tag == "location_search":
                if globLocation is not None:
                    print("Datum: ", date)
                    t, cc = getWeather(globLocation, date)
                    if t is not None:
                        result = result.replace("<location>", globLocation.address)
                        result = result.replace("<date>", "now")
                        weather_description = "fine"
                        if cc < 30:
                            weather_description = "fine"
                        elif cc < 60:
                            weather_description = "cloudy"
                        else:
                            weather_description = "bad"
                
                        weather = "%.1f" % t + "°C° with " + weather_description + " conditions"
                        result = result.replace("<weather>", weather)

                    else:
                        result = "Could not find weather"
                else:
                    result = "Could not find weather"
            break
    return result


def chatbot_response(msg):
    global errorMessage
    errorMessage = ""
    date = getDate(msg)
    ints = predict_class(msg, model)
    res  = getResponse(ints, intents, date)
    if len(errorMessage) > 0:
        return errorMessage
    return res



## 2 Part User Interface
The next part is for the handling of the graphical user interface (GUI).

The first two functions are to use the user input for the NLP logic.

The function sendText() is just an alternative function to call the main send() function, used by an enter in the input.

In [ ]:
# called by a return in the input field
# calls just the send() function
def sendText():
    send()

# This function moves a valid input into the chat field and calls the bot.
# When the bot created a return value it is also inserted into the chat field.
def send():
    msg = entryBox.get("1.0",'end-1c').strip()
    entryBox.delete("0.0",END)

    if msg != '':
        chatLog.config(state=NORMAL)
        chatLog.insert(END, "You: " + msg + '\n\n')
        chatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        chatLog.insert(END, "Bot: " + res + '\n\n')
            
        chatLog.config(state=DISABLED)
        chatLog.yview(END)

# Real Program begin where we first define a window we would like to use for user communication
window = tk.Tk()
# We give the window a title
window.title("Lueni's Chatbot")
# And a size
window.geometry('400x500')
# Which we not want to be resizable
window.resizable(width=FALSE, height=FALSE)

# Create Chat view where alle the chat history is shown
chatLog = Text(window, bd=0, bg="lightblue", height="8", width="50", font="Arial",)
# We show an first welcome message
chatLog.insert(END, "Bot: Welcome to Lueni's Chatbot. Please ask me something." + '\n\n')
# The text view should not be editable
chatLog.config(state=DISABLED )
# We turn off the binding so the Chatlog view is not selectable
chatLog.bindtags((str(chatLog), str(window), "all"))
chatLog.yview(END)

# Bind scrollbar to Chat window
scrollbar = Scrollbar(window, command=chatLog.yview, cursor="arrow")
chatLog['yscrollcommand'] = scrollbar.set

# Create the box to enter message
# The binding of the return key to the sendText function does not work on a Mac.
# There should be a solution for that but this is actually not important for this example program.
entryBox = Text(window, bd=0, bg="white", width="29", height="5", font="Arial")
entryBox.bind('<Return>', sendText)

#Create Button to send message
sendButton = Button(window, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg="#000000",
                    command=send )

#Place all components on the window
scrollbar.place(x=376,y=6, height=386)
chatLog.place(x=6,y=6, height=386, width=370)
entryBox.place(x=6, y=401, height=90, width=265)
sendButton.place(x=272, y=402, height=88, width=120)

# We set the focus to the editing field so the user
entryBox.focus_set()

# The next statement creates the window and starts a main loop of waiting for an input and reaction to that
window.mainloop()